<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-23 13:33:06
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 22.63 K (0.16%)
Current PnL: -33.19 L (-20.75%)
CY Booked + Current PnL: -17.51 L (-10.95%)
-------------------
Total profit:  1.33 L
Total loss:  -34.52 L
-------------------
Total Booked + Current PnL: 9.47 L (7.12%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.1%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.81 L (70.66%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.50,4.32,16.54,21.57,16906.0,4230.0,102210.0,397.04,-10.12,60.0,H-SC,6.14,167.0,0.25,0.74,49.16,XR,ATH,FINANCE
18,COALINDIA,-0.52,3.62,13.57,17.67,21156.0,5440.0,155900.0,478.51,24.82,52.0,L-LC,11.36,185.0,0.26,1.12,25.15,XY25,ATH,MINING
36,ICICIGI,1.38,3.98,14.78,19.34,30752.0,7958.0,208067.0,2252.93,-12.48,63.0,X-MC,1.93,66.0,0.26,1.50,20.55,X40,ATH,INSURANCE
77,TTKPRESTIG,-0.40,-30.82,44.68,0.09,31150.0,-31059.0,69718.0,770.00,61.79,30.0,M-SC,10.07,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
51,MEDANTA,2.68,-3.12,28.07,24.08,34524.0,-3958.0,122992.0,1486.00,-3.77,54.0,X-SC,9.79,87.0,-0.11,0.89,12.65,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TRIDENT,4.11,-28.83,80.59,28.51,52910.0,-26602.0,65653.0,48.00,-8.51,50.0,M-SC,8.45,223.0,-0.50,0.47,12.34,XR,NTT,TEXTILES
10,BAJAJHLDNG,3.08,3.12,25.31,29.22,49623.0,5936.0,196061.0,14451.52,-4.76,65.0,X-LC,11.98,28.0,0.12,1.41,9.21,X40,ATH,FINANCE
51,MEDANTA,2.68,-3.12,28.07,24.08,34524.0,-3958.0,122992.0,1486.00,-3.77,54.0,X-SC,9.79,87.0,-0.11,0.89,12.65,XY24,NTT,HEALTHCARE
50,MASFIN,2.50,4.32,16.54,21.57,16906.0,4230.0,102210.0,397.04,-10.12,60.0,H-SC,6.14,167.0,0.25,0.74,49.16,XR,ATH,FINANCE
27,GILLETTE,2.41,2.18,26.95,29.72,69919.0,5544.0,259440.0,10978.86,-21.51,58.0,X-SC,10.74,83.0,0.08,1.87,14.20,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,-3.34,-32.31,95.06,32.03,84826.0,-42601.0,89234.0,9161.00,-15.32,31.0,H-SC,8.63,158.0,-0.50,0.64,0.00,OX40N,NTT,IT
1,ABB,-3.16,6.38,36.84,45.57,102527.0,16685.0,278304.0,7934.00,-31.59,58.0,H-MC,6.68,125.0,0.16,2.00,23.59,AR,NTT,ELECTRICAL
23,DIXON,-3.16,-25.66,76.57,31.26,131357.0,-59216.0,171552.0,18931.72,-66.76,39.0,X-MC,24.74,54.0,-0.45,1.24,5.28,X40N,ATH,IT
69,SURYODAY,-2.95,-32.50,82.28,23.03,99452.0,-58201.0,120870.0,216.00,31.58,26.0,H-SC,4.53,166.0,-0.59,0.87,19.93,XR,NTT,BANKS
30,HAPPSTMNDS,-2.45,-54.08,302.59,84.85,178068.0,-69317.0,58848.0,1480.71,-45.32,32.0,H-SC,25.98,152.0,-0.39,0.42,0.00,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.37,2.68,102.49,107.91,155376.0,3957.0,151601.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.05,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.40,-30.82,44.68,0.09,31150.0,-31059.0,69718.0,770.00,61.79,30.0,M-SC,10.07,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
66,SIS,1.01,-31.15,76.12,21.26,58193.0,-34583.0,76449.0,528.00,1794.30,33.0,H-SC,4.59,168.0,-0.59,0.55,5.39,OX40N,NTT,MISC
72,TATAELXSI,-3.34,-32.31,95.06,32.03,84826.0,-42601.0,89234.0,9161.00,-15.32,31.0,H-SC,8.63,158.0,-0.50,0.64,0.00,OX40N,NTT,IT
40,INDIGOPNTS,0.63,-32.52,48.23,0.02,56809.0,-56771.0,117788.0,1408.00,79.69,28.0,M-SC,10.36,222.0,-1.00,0.85,2.45,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,-0.76,20.1,20.9,45.2,48129.0,38538.0,230280.0,1856.0,37.53,63.0,X-MC,6.67,79.0,0.8,1.66,28.26,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.52,3.62,13.57,17.67,21156.0,5440.0,155900.0,478.51,24.82,52.0,L-LC,11.36,185.0,0.26,1.12,25.15,XY25,ATH,MINING
26,FINCABLES,2.37,2.68,102.49,107.91,155376.0,3957.0,151601.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.05,OX40N,ATH,CABLES
50,MASFIN,2.50,4.32,16.54,21.57,16906.0,4230.0,102210.0,397.04,-10.12,60.0,H-SC,6.14,167.0,0.25,0.74,49.16,XR,ATH,FINANCE
1,ABB,-3.16,6.38,36.84,45.57,102527.0,16685.0,278304.0,7934.00,-31.59,58.0,H-MC,6.68,125.0,0.16,2.00,23.59,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.50,4.32,16.54,21.57,16906.0,4230.0,102210.0,397.04,-10.12,60.0,H-SC,6.14,167.0,0.25,0.74,49.16,XR,ATH,FINANCE
1,ABB,-3.16,6.38,36.84,45.57,102527.0,16685.0,278304.0,7934.00,-31.59,58.0,H-MC,6.68,125.0,0.16,2.00,23.59,AR,NTT,ELECTRICAL
26,FINCABLES,2.37,2.68,102.49,107.91,155376.0,3957.0,151601.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.05,OX40N,ATH,CABLES
39,INDIAMART,1.19,-4.96,117.51,106.72,137745.0,-6116.0,117220.0,4810.62,-57.23,50.0,H-SC,7.95,140.0,-0.04,0.84,17.26,AR,ATH,MISC
85,ZYDUSLIFE,0.31,-4.72,42.35,35.63,84562.0,-9900.0,199674.0,1286.17,-17.12,51.0,H-MC,5.08,120.0,-0.12,1.44,11.16,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.96,-9.68,48.64,34.24,137450.0,-30301.0,282587.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
8,AWL,-0.89,-36.23,146.51,57.19,304690.0,-118162.0,207965.0,485.00,-63.54,24.0,X-SC,17.89,81.0,-0.39,1.50,0.00,XY24,NTT,FMCG
33,HCLTECH,-0.89,-7.61,34.03,23.83,76064.0,-18403.0,223521.0,1908.19,-2.81,24.0,X-LC,3.85,8.0,-0.24,1.61,7.24,X40,ATH,IT
73,TCS,-0.51,-23.40,61.33,23.58,203241.0,-101239.0,331390.0,4311.59,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.00,X40,ATH,IT
53,PGHH,-0.18,-13.98,53.46,32.01,98195.0,-29840.0,183680.0,17616.87,-41.76,31.0,X-MC,9.56,57.0,-0.30,1.32,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,2.21,3.56,16.61,20.77,40277.0,8342.0,242486.0,1644.00,-8.17,57.0,X-MC,0.62,64.0,0.21,1.75,11.23,X40N,NTT,BREWERIES
21,DABUR,0.30,-0.05,43.77,43.70,108530.0,-112.0,247956.0,735.00,-9.31,50.0,X-MC,1.31,70.0,-0.00,1.79,15.00,XY24,BTT,FMCG
36,ICICIGI,1.38,3.98,14.78,19.34,30752.0,7958.0,208067.0,2252.93,-12.48,63.0,X-MC,1.93,66.0,0.26,1.50,20.55,X40,ATH,INSURANCE
65,SIEMENS,1.78,-12.32,43.16,25.51,70420.0,-22935.0,163160.0,4671.50,37.11,60.0,H-LC,2.01,51.0,-0.33,1.18,20.24,AR,ATH,ELECTRICAL
75,TMPV,0.36,-15.70,58.17,33.33,159763.0,-51151.0,274649.0,600.00,-82.53,59.0,X-LC,2.69,4.0,-0.32,1.98,12.28,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.24,-24.74,106.58,55.47,195148.0,-60186.0,183100.0,181.50,-35.39,34.0,X-MC,17.62,60.0,-0.31,1.32,0.0,X40N,ATH,FINANCE
8,AWL,-0.89,-36.23,146.51,57.19,304690.0,-118162.0,207965.0,485.00,-63.54,24.0,X-SC,17.89,81.0,-0.39,1.50,0.0,XY24,NTT,FMCG
53,PGHH,-0.18,-13.98,53.46,32.01,98195.0,-29840.0,183680.0,17616.87,-41.76,31.0,X-MC,9.56,57.0,-0.30,1.32,0.0,X40,ATH,FMCG
42,INFY,-1.96,-9.68,48.64,34.24,137450.0,-30301.0,282587.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.0,X40,NTT,IT
73,TCS,-0.51,-23.40,61.33,23.58,203241.0,-101239.0,331390.0,4311.59,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.0,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.08,-31.79,109.42,42.84,48516.0,-20667.0,44339.0,424.00,-55.37,45.0,X-SC,17.87,80.0,-0.43,0.32,6.65,XY24,NTT,MISC
58,RELAXO,-0.39,-53.82,229.83,52.31,154057.0,-78129.0,67031.0,1176.00,-50.83,39.0,X-SC,14.07,92.0,-0.51,0.48,0.49,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.26,-47.49,157.49,35.20,106403.0,-61108.0,67562.0,2096.00,-11.48,33.0,X-SC,21.80,91.0,-0.57,0.49,0.26,X40,NTT,FOOTWEAR
51,MEDANTA,2.68,-3.12,28.07,24.08,34524.0,-3958.0,122992.0,1486.00,-3.77,54.0,X-SC,9.79,87.0,-0.11,0.89,12.65,XY24,NTT,HEALTHCARE
35,HONAUT,1.59,-22.92,85.94,43.32,107889.0,-37332.0,125540.0,58357.33,-32.37,41.0,X-SC,11.55,90.0,-0.35,0.90,1.93,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.64,-14.42,39.12,19.06,113882.0,-49056.0,291110.0,452.00,-53.09,49.0,X-LC,17.91,1.0,-0.43,2.10,5.40,X40,NTT,FMCG
73,TCS,-0.51,-23.40,61.33,23.58,203241.0,-101239.0,331390.0,4311.59,-33.46,26.0,X-LC,6.18,2.0,-0.50,2.39,0.00,X40,ATH,IT
42,INFY,-1.96,-9.68,48.64,34.24,137450.0,-30301.0,282587.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
75,TMPV,0.36,-15.70,58.17,33.33,159763.0,-51151.0,274649.0,600.00,-82.53,59.0,X-LC,2.69,4.0,-0.32,1.98,12.28,XY24,NTT,AUTO
33,HCLTECH,-0.89,-7.61,34.03,23.83,76064.0,-18403.0,223521.0,1908.19,-2.81,24.0,X-LC,3.85,8.0,-0.24,1.61,7.24,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.98,-32.30,96.30,32.89,50332.0,-24940.0,52266.0,1800.00,-345.80,50.0,L-MC,11.28,258.0,-0.50,0.38,43.94,XR,NTT,BANKS
67,SONACOMS,1.19,-6.85,49.32,39.09,46474.0,-6930.0,94229.0,804.02,-26.14,66.0,M-MC,9.15,193.0,-0.15,0.68,33.30,AR,ATH,AUTO
50,MASFIN,2.50,4.32,16.54,21.57,16906.0,4230.0,102210.0,397.04,-10.12,60.0,H-SC,6.14,167.0,0.25,0.74,49.16,XR,ATH,FINANCE
18,COALINDIA,-0.52,3.62,13.57,17.67,21156.0,5440.0,155900.0,478.51,24.82,52.0,L-LC,11.36,185.0,0.26,1.12,25.15,XY25,ATH,MINING
37,ICICIPRULI,2.00,9.93,18.92,30.73,40974.0,19557.0,216562.0,790.00,-19.66,60.0,X-MC,4.18,78.0,0.48,1.56,23.96,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-0.86,-49.12,186.33,45.70,167639.0,-86846.0,89969.0,26.40,-6.64,73.0,M-SC,19.05,204.0,-0.52,0.65,48.23,XY24,NTT,TRAVEL
7,ATULAUTO,2.21,-16.21,73.73,45.57,121781.0,-31960.0,165172.0,844.00,3575.68,56.0,M-SC,3.93,248.0,-0.26,1.19,25.00,XY24,NTT,AUTO
1,ABB,-3.16,6.38,36.84,45.57,102527.0,16685.0,278304.0,7934.00,-31.59,58.0,H-MC,6.68,125.0,0.16,2.00,23.59,AR,NTT,ELECTRICAL
67,SONACOMS,1.19,-6.85,49.32,39.09,46474.0,-6930.0,94229.0,804.02,-26.14,66.0,M-MC,9.15,193.0,-0.15,0.68,33.30,AR,ATH,AUTO
82,VOLTAS,-0.76,20.10,20.90,45.20,48129.0,38538.0,230280.0,1856.00,37.53,63.0,X-MC,6.67,79.0,0.80,1.66,28.26,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.63
1,25,45.39
2,50,77.32


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.12
MC    30.10
LC    24.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.02
X40      24.06
X40N     14.76
AR        8.94
XY25      8.76
XR        8.49
OX40N     7.11
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.69
H-SC    22.13
X-LC    21.43
X-SC    11.55
M-SC    10.70
H-MC     4.96
M-MC     2.07
H-LC     1.18
L-LC     1.12
M-LC     1.03
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.59,-14.01,52.57
FINANCE,13.09,-14.89,60.11
IT,11.46,-37.91,107.02
MISC,6.87,-35.93,91.54
ELECTRICAL,6.21,-8.76,48.44
PAINTS,5.04,-31.95,51.56
INSURANCE,4.86,-0.61,35.04
PHARMA,4.10,-3.14,35.61
AUTO,3.85,-16.86,61.42


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3376880.0,21
X40,1381871.0,15
AR,1330002.0,10
XR,1314539.0,12
X40N,1070103.0,10
OX40N,804915.0,10
XY25,400085.0,6
SR,302496.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3720685.0,24
M-SC,1429931.0,14
X-MC,1362953.0,15
X-LC,1298774.0,12
X-SC,1200791.0,10
H-MC,389838.0,3
L-SC,188595.0,2
M-LC,149114.0,1
M-MC,98302.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1265382.0      6
           AR         951461.0      5
           XR         845730.0      7
M-SC       XY24       825306.0      6
X-LC       X40        670172.0      6
X-SC       XY24       616572.0      4
X-MC       X40        535377.0      7
           X40N       430424.0      4
X-SC       X40N       407897.0      4
H-SC       OX40N      355616.0      4
M-SC       OX40N      353553.0      5
H-SC       SR         302496.0      2
X-LC       XY24       270658.0      2
           X40N       231782.0      2
H-MC       XY24       202749.0      1
X-MC       XY25       200939.0      2
           XY24       196213.0      2
H-MC       AR         187089.0      2
M-SC       XR         176514.0      2
X-SC       X40        176322.0      2
M-LC       XR         149114.0      1
X-LC       XY25       126162.0      2
L-SC       OX40N       95746.0      1
           XR          92849.0      1
M-SC       AR          74558.0      1
H-LC       AR          70420.0      1
M-MC       XY25        51828.0      1
L-MC       XR          50332.0      1
M-MC       AR          46474.0      1
L-LC       XY25        21156.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
